In [6]:
import numpy as np
import matplotlib.pyplot as plt
# from data import x_val, y_val
import tensorflow as tf
from tensorflow import keras
# from keras_unet_collection.models import unet_2d

seed = 64
batch_size = 32
tf.random.set_seed(seed)

tf.__version__

'2.4.1'

In [12]:
# 이미지 전처리로 스케일링을 합니다
img_datagen = keras.preprocessing.image.ImageDataGenerator( rescale = 1.0/255. )



# x_val
x_val_dir = '/media/visbic/MGTEC/water_seg/water_segmentation/Validation/[원천]validate_water_data'
x_val_generator = img_datagen.flow_from_directory(x_val_dir, batch_size=300, shuffle=False, seed=seed, class_mode=None, color_mode="grayscale")#"grayscale"

x_val = x_val_generator.next()
print("x_val.shape :", x_val.shape)



# from tensorflow.python.ops.numpy_ops import np_config
# np_config.enable_numpy_behavior()

# y_val
y_val_dir = '/media/visbic/MGTEC/water_seg/water_segmentation/Validation/[라벨]validate_water_labeling'
y_val_generator = img_datagen.flow_from_directory(y_val_dir, batch_size=300, shuffle=False, seed=seed, class_mode=None, color_mode="grayscale")#"grayscale"

y_val = y_val_generator.next()
print("y_val.shape :", y_val.shape)

ohy_val = tf.one_hot(
    y_val, 2, on_value=None, off_value=None, axis=None, dtype=None, name=None
)
print("resahpe 전 ohy_val.shape :", ohy_val.shape)
ohy_val = ohy_val.numpy()
ohy_val = ohy_val.reshape(ohy_val.shape[0], 256, 256, 2)
print("resahpe 후 ohy_val.shape :", ohy_val.shape)

Found 300 images belonging to 1 classes.
x_val.shape : (300, 256, 256, 1)
Found 300 images belonging to 1 classes.
y_val.shape : (300, 256, 256, 1)
resahpe 전 ohy_val.shape : (300, 256, 256, 1, 2)
resahpe 후 ohy_val.shape : (300, 256, 256, 2)


In [15]:
from keras_unet_collection.activations import GELU
from keras_unet_collection.losses import iou_seg

# /media/visbic/MGTEC/water_seg/water_segmentation/unet_model/20211211-040154/10-0.9812.hdf5
model = keras.models.load_model('/media/visbic/MGTEC/water_seg/water_segmentation/unet_model/20211211-040154/10-0.9812.hdf5', 
                                custom_objects={
                                                "GELU":GELU,
                                                "iou_seg":iou_seg
                                                })

In [16]:
pred = model.predict(x_val)
pred.shape

2021-12-11 15:07:31.372555: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-11 15:07:31.389348: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3699850000 Hz
2021-12-11 15:07:51.561919: W tensorflow/core/framework/op_kernel.cc:1763] OP_REQUIRES failed at mkl_softmax_op.cc:309 : Aborted: Operation received an exception:Status: 2, message: could not create a descriptor for a softmax forward propagation primitive, in file tensorflow/core/kernels/mkl/mkl_softmax_op.cc:306


AbortedError:  Operation received an exception:Status: 2, message: could not create a descriptor for a softmax forward propagation primitive, in file tensorflow/core/kernels/mkl/mkl_softmax_op.cc:306
	 [[node unet_model/unet_output_activation/Softmax (defined at tmp/ipykernel_35466/2609307764.py:1) ]] [Op:__inference_predict_function_2973]

Function call stack:
predict_function


In [ ]:
pred2 = np.zeros((300,256,256,1))

In [ ]:
(pred[:,:,:,0] > pred[:,:,:,1]).sum()

In [ ]:
(pred[:,:,:,0] < pred[:,:,:,0]).sum()

In [ ]:
pred2 = pred[:,:,:,0]
pred2.shape

In [ ]:
pred2[pred[:,:,:,0] > 0.5] = 0

In [ ]:
pred2[pred[:,:,:,0] < 0.5] = 255

In [ ]:
def showPred(idx):
    # 본래 이미지
    plt.subplot(121)
    plt.imshow(x_val[idx].reshape(256,256), cmap='gray')
    plt.title("predicted")

    # ground truth
    plt.subplot(122)
    plt.imshow(y_val[idx].reshape(256,256), cmap='gray')
    plt.title("label")
    
    # 예측
    plt.subplot(123)
    plt.imshow(pred2[idx].reshape(256,256), cmap='gray')
    plt.title("predicted")

    plt.show()

In [ ]:
for i in range(300):
    showPred(i)